<a href="https://colab.research.google.com/github/xoyeon/hongong-mldl/blob/main/05_2_%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검증 세트
테스트 세트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다
> 모델을 만들고 나서 마지막에 딱 한 번만 사용할 것

훈련 세트를 나누어 모델 검증하기
> 이 데이터를 검증 세트(validation set)이라고 부름

In [1]:
# 데이터 불러오기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
# class 열을 타깃으로 사용, 나머지는 특성 배열에 저장
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
# 훈련 세트와 데이터 세트 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# 훈련 세트(sub)와 검증 세트(val) 만들기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [5]:
# 훈련 세트와 검증 세트의 크기 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
# sub_input, sub_target과 val_input, val_target을 사용해 모델 만들고 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증
교차 검증(cross validation)을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있음
- 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복
- 이 점수를 평균하여 최종 검증 점수를 얻음

In [7]:
# cross_validate()를 사용하여 교차검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00744867, 0.00707889, 0.00732636, 0.00724077, 0.00723457]), 'score_time': array([0.00075698, 0.00064969, 0.00068688, 0.00130177, 0.00066066]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
# 교차 검증의 최종 점수
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
# 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# 교차 검증을 할 때 훈련 세트를 섞으려면 splitter 지정. 회귀 모델일 경우 KFold 분할기 사용
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)  ## 10-폴드 교차 검증 수행
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼파라미터 튜닝
사이킷런의 GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행

In [11]:
# GridSearchCV 클래스를 임포트하고, 탐색할 매개변수와 리스트를 딕셔너리로 만듦
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
# GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체 만들기
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)

In [13]:
# gs 객체에 fit() 메서드 호출
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [14]:
# 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델 훈련
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
# 그리드 서치로 찾은 최적의 매개변수 확인
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [16]:
# 5번의 교차 검증으로 얻은 점수 출력
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
# argmax()를 사용하여 가장 큰 값의 인덱스를 추출한 후 params 키에 저장된 매개변수 출력
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


### 정리하기
1. 먼저 탐색할 매개변수 지정
2. 훈련 세트에서 그리드 서치 수행
  - 최상의 평균 검증 점수가 나오는 매개변수 조합 찾기
    - 이 조합은 그리드 서치 객체에 저장됨
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델 훈련
  - 교차 검증에 사용한 훈련 세트 아님
    - 이 모델도 그리드 서치 객체에 저장됨

In [23]:
# 조금 더 복잡한 매개변수 조합 탐색하기
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),  ## 첫 번째 매개변수 값에서 시작, 두 번째 매변수가 도달할 때까지 세 번째 매개변수를 계속 더한 배열을 만듦
          'max_depth' : range(5, 20, 1),  ## max_depth를 5에서 20까지 1씩 증가하면서 15개의 값을 만듦
          'min_samples_split' : range(2, 100, 10)
          }

In [24]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [25]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [26]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# 랜덤 서치
- 매개변수의 값이 수칠일 때 값의 범위나 가격을 미리 정하기 어려울 수 있음
- 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있음


In [27]:
# 싸이파이에서 2개의 확률 분포 클래스 임포트
from scipy.stats import uniform, randint  ## randint 정숫값, uniform 실숫값

In [28]:
# 0에서 10 사이의 범위를 갖는 randint 객체 만들고 10개의 숫자 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([6, 9, 0, 7, 3, 7, 9, 1, 8, 2])

In [29]:
# 1000개를 샘플링해서 각 숫자의 개수를 세어 보기
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([120,  87,  87,  91,  94, 101, 109, 110, 103,  98]))

In [30]:
# 0~1 사이에서 10개의 실수를 추출
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.76441734, 0.75238093, 0.52034686, 0.30387122, 0.12039769,
       0.83697913, 0.74842809, 0.13468635, 0.69010707, 0.94769592])

In [31]:
# 탐색할 매개변수의 딕셔너리 만들기
param = {'min_impurity_decrease' : uniform(0.0001, 0.001),
         'max_depth' : randint(20, 50),
         'min_samples_split' : randint(2, 25),
         'min_samples_leaf' : randint(1, 25)
         }

In [32]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [33]:
print(gs.best_params_)

{'min_samples_split': 12, 'min_impurity_decrease': 0.0005, 'max_depth': 11}


In [34]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8681935292811135


In [35]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8615384615384616
